In [14]:
import pandas as pd
import numpy as np

from orion.data import load_signal

# 1. Data

In [2]:
signal_name = 'S-1'

data = load_signal(signal_name)

data.head()

,timestamp,value
0,1222819200,-0.366359
1,1222840800,-0.394108
2,1222862400,0.403625
3,1222884000,-0.362759
4,1222905600,-0.370746


# 2. Pipeline

In [3]:
from mlblocks import MLPipeline

pipeline_name = 'tadgan'

pipeline = MLPipeline(pipeline_name)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [4]:
hyperparameters = {
    'orion.primitives.tadgan.TadGAN#1': {
        'epochs': 5,
        'verbose': True,
        'validation_split': 0.0
    }
}

pipeline.set_hyperparameters(hyperparameters)

## step by step execution

MLPipelines are compose of a squence of primitives, these primitives apply tranformation and calculation operations to the data and updates the variables within the pipeline. To view the primitives used by the pipeline, we access its `primtivies` attribute. 

The `tadgan` pipeline contains 7 primitives. we will observe how the `context` (which are the variables held within the pipeline) are updated after the execution of each primitive.

In [5]:
pipeline.primitives

['mlprimitives.custom.timeseries_preprocessing.time_segments_aggregate',
 'sklearn.impute.SimpleImputer',
 'sklearn.preprocessing.MinMaxScaler',
 'mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences',
 'orion.primitives.tadgan.TadGAN',
 'orion.primitives.tadgan.score_anomalies',
 'orion.primitives.timeseries_anomalies.find_anomalies']

### time segments aggregate
this primitive creates an equi-spaced time series by aggregating values over fixed specified interval.

* **input**: `X` which is an n-dimensional sequence of values.
* **output**:
    - `X` sequence of aggregated values, one column for each aggregation method.
    - `index` sequence of index values (first index of each aggregated segment).

In [6]:
context = pipeline.fit(data, output_=0)
context.keys()

dict_keys(['X', 'index'])

In [7]:
for i, x in list(zip(context['index'], context['X']))[:5]:
    print("entry at {} has value {}".format(i, x))

entry at 1222819200 has value [-0.36635895]
entry at 1222840800 has value [-0.39410778]
entry at 1222862400 has value [0.4036246]
entry at 1222884000 has value [-0.36275906]
entry at 1222905600 has value [-0.37074649]


### SimpleImputer
this primitive is an imputation transformer for filling missing values.
* **input**: `X` which is an n-dimensional sequence of values.
* **output**: `X` which is a transformed version of X.

In [8]:
step = 1

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

dict_keys(['index', 'X'])

### MinMaxScaler
this primitive transforms features by scaling each feature to a given range.
* **input**: `X` the data used to compute the per-feature minimum and maximum used for later scaling along the features axis.
* **output**: `X` which is a transformed version of X.

In [9]:
step = 2

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

dict_keys(['index', 'X'])

In [10]:
# after scaling the data between [-1, 1]
# in this example, no change is observed
# since the data was pre-handedly scaled

for i, x in list(zip(context['index'], context['X']))[:5]:
    print("entry at {} has value {}".format(i, x))

entry at 1222819200 has value [-0.36635895]
entry at 1222840800 has value [-0.39410778]
entry at 1222862400 has value [0.4036246]
entry at 1222884000 has value [-0.36275906]
entry at 1222905600 has value [-0.37074649]


### rolling window sequence
this primitive generates many sub-sequences of the original sequence. it uses a rolling window approach to create the sub-sequences out of time series data.

* **input**: 
    - `X` n-dimensional sequence to iterate over.
    - `index` array containing the index values of X.
* **output**:
    - `X` input sequences.
    - `y` target sequences.
    - `index` first index value of each input sequence -> renamed to `X_index`.
    - `target_index` first index value of each target sequence.

In [11]:
step = 3

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

dict_keys(['index', 'X', 'y', 'X_index', 'target_index'])

In [12]:
# after slicing X into multiple sub-sequences
# we obtain a 3 dimensional matrix X where
# the shape indicates (# slices, window size, 1)
# and similarly y is (# slices, target size)

print("X shape = {}\ny shape = {}\nindex shape = {}\ntarget index shape = {}".format(
    context['X'].shape, context['y'].shape, context['index'].shape, context['target_index'].shape))

X shape = (10049, 100, 1)
y shape = (10049, 1)
index shape = (10149,)
target index shape = (10049,)


### TadGAN
this is a reconstruction based model, namely Generative Adversarial Networks (GAN), containing multiple neural networks and cycle consistency loss. the proposed model is described in the [related paper](https://arxiv.org/pdf/2009.07769.pdf).

* **input**: 
    - `X` n-dimensional array containing the input sequences for the model.
* **output**: `y` reconstructed values -> renamed `y_hat`.

In [13]:
step = 4

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/Users/sarah/opt/anaconda3/envs/orion/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/Users/sarah/opt/anaconda3/envs/orion/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/Users/sarah/opt/anaconda3/envs/orion/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/Users/sarah/opt/anaconda3/envs/orion/lib/python3.6/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch: 1/5, [Dx loss: [-2.6776628  -4.5917287   0.9348483   0.09792167]] [Dz loss: [-1.0969741  -1.4592503  -0.78547704  0.11477532]] [G loss: [ 2.9390972  -0.9107458   1.4552648   0.23945776]]
Epoch: 2/5, [Dx loss: [-3.6530893  -5.466679    1.3018161   0.05117736]] [Dz loss: [-3.2299697  -3.0031188  -1.3129394   0.10860875]] [G loss: [ 2.799485   -1.2751291   1.7046444   0.23699693]]
Epoch: 3/5, [Dx loss: [-3.4280794  -5.222771    1.3065965   0.04880931]] [Dz loss: [-2.2623312  -2.473338   -0.3233766   0.05343825]] [G loss: [ 1.7164162  -1.3127668   0.7973495   0.22318335]]
Epoch: 4/5, [Dx loss: [-3.4212792  -5.419557    1.5358766   0.04624021]] [Dz loss: [-1.5228959  -0.24059121 -1.700906    0.04186013]] [G loss: [ 2.9546404  -1.5201428   2.318647    0.21561353]]
Epoch: 5/5, [Dx loss: [-3.4299202  -5.2663007   1.4025416   0.04338401]] [Dz loss: [-0.9210284   0.37030175 -1.9520515   0.06607217]] [G loss: [ 3.5728276  -1.3929796   2.7340674   0.22317404]]


dict_keys(['index', 'X_index', 'target_index', 'X', 'y', 'y_hat', 'critic'])

In [15]:
# from y_hat we compute the
# flattened value of each 
# point using the median 
# reconstructed value

def unroll_ts(y_hat):
    predictions = list()
    pred_length = y_hat.shape[1]
    num_errors = y_hat.shape[1] + (y_hat.shape[0] - 1)

    for i in range(num_errors):
            intermediate = []

            for j in range(max(0, i - num_errors + pred_length), min(i + 1, pred_length)):
                intermediate.append(y_hat[i - j, j])

            if intermediate:
                predictions.append(np.median(np.asarray(intermediate)))

    return np.asarray(predictions[pred_length-1:])

for i, y, y_hat in list(zip(context['target_index'], context['y'], unroll_ts(context['y_hat'])))[:5]:
    print("entry at {} has value {}, predicted value {}".format(i, y, y_hat))

entry at 1224979200 has value [1.], predicted value -0.4282296895980835
entry at 1225000800 has value [-0.34946279], predicted value -0.4239683151245117
entry at 1225022400 has value [-0.37232052], predicted value -0.44071683287620544
entry at 1225044000 has value [1.], predicted value -0.44799137115478516
entry at 1225065600 has value [-0.24757408], predicted value -0.45670443773269653


### score anomalies

this primitive computes an array of anomaly scores based on a combination of reconstruction error and critic output.

* **input**: 
    - `y` ground truth -> renamed to `X`.
    - `y_hat` predicted values. Each timestamp has multiple predictions.
    - `critic` critic score. Each timestamp has multiple critic scores.  
    - `index` time index for each `y` (start position of the window).
* **output**: 
    - `errors` array of scores.
    - `true_index`  time index of scores.
    - `true` ground truth.
    - `predictions` predicted sequence.

In [16]:
step = 5

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

dict_keys(['index', 'X_index', 'target_index', 'y_hat', 'critic', 'X', 'y', 'errors', 'true_index', 'true', 'predictions'])

In [17]:
for i, e in list(zip(context['target_index'], context['errors']))[:5]:
    print("entry at {} has error value {:.3f}".format(i, e))

entry at 1224979200 has error value 1.571
entry at 1225000800 has error value 1.569
entry at 1225022400 has error value 1.567
entry at 1225044000 has error value 1.565
entry at 1225065600 has error value 1.562


### find anomalies
this primitive extracts anomalies from sequences of errors following the approach explained in the [related paper](https://arxiv.org/pdf/1802.04431.pdf).

* **input**: 
    - `errors` array of errors.
    - `target_index` array of indices of errors.
* **output**: `y` array containing start-index, end-index, score for each anomalous sequence that was found.

In [18]:
step = 6

context = pipeline.fit(**context, output_=step, start_=step)
context.keys()

dict_keys(['index', 'X_index', 'target_index', 'y_hat', 'critic', 'errors', 'true_index', 'true', 'predictions', 'X', 'y'])

In [19]:
pd.DataFrame(context['y'], columns=['start', 'end', 'severity'])

,start,end,severity
0,1.248264e+09,1.250640e+09,0.152003
1,1.265544e+09,1.267963e+09,0.146268
2,1.349741e+09,1.352333e+09,0.265520
3,1.401516e+09,1.405361e+09,2.304467
